In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from utils import *

# helps in text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# helps in LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping


%matplotlib inline

In [16]:
parent_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [17]:

df_train = pd.read_csv(os.path.join(parent_dir, 'Data\AugmentedData\english_train_augmented.csv'))
df_dev = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_dev_preprocess.csv'))
df_test = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_test_preprocess.csv'))

In [18]:
df_train = df_train[['augmented_text', 'label']]
df_dev = df_dev[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text', 'label']]

In [19]:
label_replacement = {
    'Hope_speech': 0,
    'Non_hope_speech': 1,
    'not-English': 2,
}

df_train['label'] = df_train['label'].replace(label_replacement)
df_test['label'] = df_test['label'].replace(label_replacement)
df_dev['label'] = df_dev['label'].replace(label_replacement)

# Drop rows with label 2
df_train = df_train[df_train['label'] != 2]
df_test = df_test[df_test['label'] != 2]
df_dev = df_dev[df_dev['label'] != 2]

In [20]:
X_train, y_train = df_train['augmented_text'].to_numpy(), df_train['label'].to_numpy()
X_dev, y_dev = df_dev['preprocessed_text'].to_numpy(), df_dev['label'].to_numpy()
X_test, y_test = df_test['preprocessed_text'].to_numpy(), df_test['label'].to_numpy()

In [21]:
t = Tokenizer()
t.fit_on_texts(X_train)

In [22]:
encoded_train = t.texts_to_sequences(X_train)
encoded_dev = t.texts_to_sequences(X_dev)
encoded_test = t.texts_to_sequences(X_test)

In [23]:
max_length = 8
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_dev = pad_sequences(encoded_dev, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [24]:
vocab_size = len(t.word_index) + 1

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 24, input_length=max_length))
model.add(LSTM(24, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8, 24)             521064    
                                                                 
 lstm_1 (LSTM)               (None, 24)                4704      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 525,793
Trainable params: 525,793
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# fit the model
model.fit(
    x=padded_train,
    y=y_train,
    epochs=100,
    validation_data=(padded_dev, y_dev), verbose=1,
    callbacks=[early_stop]
)

Epoch 1/100
1324/1324 [==============================] - 9s 5ms/step - loss: 0.4692 - accuracy: 0.7725 - val_loss: 0.4436 - val_accuracy: 0.7951
Epoch 2/100
1324/1324 [==============================] - 8s 6ms/step - loss: 0.3518 - accuracy: 0.8467 - val_loss: 0.4261 - val_accuracy: 0.8054
Epoch 3/100
1324/1324 [==============================] - 8s 6ms/step - loss: 0.3111 - accuracy: 0.8681 - val_loss: 0.3790 - val_accuracy: 0.8381
Epoch 4/100
1324/1324 [==============================] - 8s 6ms/step - loss: 0.2781 - accuracy: 0.8842 - val_loss: 0.3726 - val_accuracy: 0.8441
Epoch 5/100
1324/1324 [==============================] - 7s 5ms/step - loss: 0.2540 - accuracy: 0.8960 - val_loss: 0.3884 - val_accuracy: 0.8342
Epoch 6/100
1324/1324 [==============================] - 6s 5ms/step - loss: 0.2365 - accuracy: 0.9034 - val_loss: 0.3640 - val_accuracy: 0.8546
Epoch 7/100
1324/1324 [==============================] - 6s 5ms/step - loss: 0.2201 - accuracy: 0.9121 - val_loss: 0.4216 - val_ac

In [26]:
train_preds = model.predict(padded_train)
dev_preds = model.predict(padded_dev)
test_preds = model.predict(padded_test)

train_preds = np.where(train_preds > 0.5, 1, 0)
dev_preds = np.where(dev_preds > 0.5, 1, 0)
test_preds = np.where(test_preds > 0.5, 1, 0)

train_preds = train_preds.flatten()
dev_preds = dev_preds.flatten()
test_preds = test_preds.flatten()

89/89 [==============================] - 0s 2ms/step


In [27]:
computeAllScores(train_preds, dev_preds, test_preds, True, True)

Accuracy Train:  0.9340415486307838
Accuracy Dev:  0.833861316437874
Accuracy Test:  0.8304607808652832
Weighted F1 Train:  0.9339944893217119
Weighted F1 Dev:  0.8538250329701035
Weighted F1 Test:  0.854308848966864
Macro F1 Train:  0.9339470761832485
Macro F1 Dev:  0.6415256955880152
Macro F1 Test:  0.6290023023458804
Micro F1 Train:  0.9340415486307838
Micro F1 Dev:  0.833861316437874
Micro F1 Test:  0.8304607808652832
Weighted Recall Train:  0.9340415486307838
Weighted Recall Dev:  0.833861316437874
Weighted Recall Test:  0.8304607808652832
Macro Recall Train:  0.9336600914447455
Macro Recall Dev:  0.6977537609049069
Macro Recall Test:  0.6956182028538372
Micro Recall Train:  0.9340415486307838
Micro Recall Dev:  0.833861316437874
Micro Recall Test:  0.8304607808652832
Confusion Matrix Train: 
[[20584   998]
 [ 1796 18982]]
Confusion Matrix Dev: 
[[ 144  128]
 [ 344 2225]]
Confusion Matrix Test: 
[[ 133  117]
 [ 365 2228]]
